In [1]:
import sys
import numpy as np
import pandas.io.sql
import pyodbc
import pandas as pd

#from IPython.core.debugger import set_trace

In [2]:
sys.path.append('dmbtools_1_0_1_0.zip')
sys.path.append('dmbtools_1_0_1_0')

In [3]:
#from dmbtools.wellbore.survey import position_log
from dmbtools.wellbore.horizontal_stats_pak import h_stats, h_stats_meta
from dmbtools.geodetic.projection import GridConvergence
from dmbtools.geodetic import convert as cvt
from dmbtools.geodetic import projection as prj
from dmbtools.math import utils as mutils

#### Setup our connection to the GGX database...

In [4]:
cnxn_str = 'DRIVER={0};HOST={1};DATABASE={2};DBF={3};UID={4};PWD={5}'.format('SQL Anywhere 17', 'GGX07-MID-PRD.conchoresources.com:2638', 'GGX007_DBBU-GGX_Proj', 'E:\GGX_Proj\GGX007_DBBU\gxdb.db', 'dba', 'sql')
cnxn_str

'DRIVER=SQL Anywhere 17;HOST=GGX07-MID-PRD.conchoresources.com:2638;DATABASE=GGX007_DBBU-GGX_Proj;DBF=E:\\GGX_Proj\\GGX007_DBBU\\gxdb.db;UID=dba;PWD=sql'

In [5]:
cnxn = pyodbc.connect(cnxn_str)

In [6]:
#sorted(list(pd.read_sql("select table_name from sys.systab where table_type_str = 'BASE'", cnxn)['table_name']))

Create the list of UWI's and get the survey stations

In [7]:
#well_list_str = ','.join(["'" + str(uwi) + "'" for uwi in well_list['UWI']])
well_list_str = "'300150011700', '300150013401', '300150070300', '300150071701'" # for now these two because the project above does have any of the UWI in the Hound Dog area.
well_list_long = "'300150011700', '300150013401', '429980051251', '429991861452'" # list of wells with long laterals

In [8]:
#sql_stations = 'select * from WELL_DIR_SRVY_STATION WHERE UWI IN ({0})'.format(well_list_str)
#sql_stations = 'select * from WELL_DIR_SRVY_STATION'
sql_stations = 'select * from WELL_DIR_SRVY_STATION WHERE UWI IN ({0})'.format(well_list_long)
sql_wells = 'select * from WELL WHERE UWI IN ({0})'.format(well_list_long)

In [9]:
sql_wells

"select * from WELL WHERE UWI IN ('300150011700', '300150013401', '429980051251', '429991861452')"

In [10]:
well_data = pandas.read_sql(sql_wells, cnxn)

In [11]:
well_data.head()

UWI PRIMARY_SOURCE WELL_GOVT_ID           WELL_NAME WELL_NUMBER  \
0  300150011700           None         None        WERTHIEM A L           1   
1  300150013401           None         None  MCIVER FEDERAL COM           2   
2  429980051251           None         None        BLACK CANYON       9572H   
3  429991861452           None         None  WILLIWAW UNIT PLAN       9201H   

  LEGAL_SURVEY_TYPE SURFACE_NODE_ID BASE_NODE_ID  SURFACE_LATITUDE  \
0          CONGRESS            None         None          32.61497   
1          CONGRESS            None         None          32.37169   
2             TEXAS            None         None          31.00246   
3             TEXAS            None         None          31.08137   

   SURFACE_LONGITUDE  ...                                   GX_DEV_WELL_BLOB  \
0         -104.46575  ...  b'\x11Minimum Curvature\x03\x00\x00\x00a\x00\x...   
1         -104.41557  ...  b'\x11Minimum Curvature\x03\x00\x00\x00\x8b\x0...   
2         -103.42931  ...  b'\x11Minimum Curvature\x03\x00\x00\x00\xed\x0...   
3         -103.46939  ...  b'\x11Minimum Curvature\x03\x00\x00\x00\xc4\x0...   

         ROW_CHANGED_DATE GX_MAG_DECLINATION GX_PERCENT_ALLOCATION  \
0 2019-09-18 17:40:51.556               None                 100.0   
1 2019-06-16 17:24:05.563               None                 100.0   
2 2019-11-12 07:46:31.072               None                 100.0   
3 2020-02-04 14:24:11.756               None                 100.0   

            COMMON_WELL_NAME           ORIGINAL_OPERATOR GGX_INTERNAL_STATUS  \
0             WERTHIEM A L 1               STANOLIND OIL                None   
1       MCIVER FEDERAL COM 2  CHI OPERATING INCORPORATED                None   
2  BLACK CANYON 9572H (PLAN)                        None                None   
3        WILLIWAW UNIT 9201H                        None                None   

    GX_WSN GX_PROPOSED_FLAG                     GX_POINT  
0   128331            False  Point (-104.46575 32.61497)  
1      114            False  Point (-104.41557 32.37169)  
2   127621            False  Point (-103.42931 31.00246)  
3  5265410            False  Point (-103.46939 31.08137)  

[4 rows x 111 columns]

In [12]:
stations_data = pandas.read_sql(sql_stations, cnxn)

In [13]:
stations_data.head()

UWI   SOURCE     SURVEY_ID  STATION_MD STATION_ID STATION_MD_OUOM  \
0  300150011700  DEFAULT  300150011700         0.0       None            None   
1  300150011700  DEFAULT  300150011700      1151.0       None            None   
2  300150011700  DEFAULT  300150011700      1341.0       None            None   
3  300150011700  DEFAULT  300150011700      1529.0       None            None   
4  300150011700  DEFAULT  300150011700      1717.0       None            None   

   INCLINATION INCLINATION_OUOM  AZIMUTH AZIMUTH_OUOM  ...  X_OFFSET  \
0          0.0             None   359.93         None  ...      0.00   
1          0.4             None   354.63         None  ...     -0.38   
2          0.4             None   346.83         None  ...     -0.59   
3          0.3             None   353.73         None  ...     -0.79   
4          0.1             None   324.13         None  ...     -0.94   

  X_OFFSET_OUOM  EW_DIRECTION Y_OFFSET Y_OFFSET_OUOM  NS_DIRECTION  \
0          None          None     0.00          None          None   
1          None          None     4.00          None          None   
2          None          None     5.31          None          None   
3          None          None     6.43          None          None   
4          None          None     7.06          None          None   

            ROW_CHANGED_DATE GX_STATION_LATITUDE GX_STATION_LONGITUDE  \
0 2019-09-18 17:40:51.275000            32.61497           -104.46575   
1 2019-09-18 17:40:51.275001            32.61498           -104.46575   
2 2019-09-18 17:40:51.291000            32.61498           -104.46575   
3 2019-09-18 17:40:51.291001            32.61499           -104.46575   
4 2019-09-18 17:40:51.291002            32.61499           -104.46575   

   GX_CLOSURE  
0        None  
1        None  
2        None  
3        None  
4        None  

[5 rows x 22 columns]

In [14]:
max_mds = stations_data.groupby(['UWI'])['STATION_MD'].max()

In [15]:
max_mds

UWI
300150011700    10486.00
300150013401    11286.00
429980051251    22823.04
429991861452    18993.80
Name: STATION_MD, dtype: float64

In [16]:
stations_data['UWI'].drop_duplicates()

0      300150011700
96     300150013401
234    429980051251
470    429991861452
Name: UWI, dtype: object

In [17]:
list(stations_data.columns)

['UWI',
 'SOURCE',
 'SURVEY_ID',
 'STATION_MD',
 'STATION_ID',
 'STATION_MD_OUOM',
 'INCLINATION',
 'INCLINATION_OUOM',
 'AZIMUTH',
 'AZIMUTH_OUOM',
 'STATION_TVD',
 'STATION_TVD_OUOM',
 'X_OFFSET',
 'X_OFFSET_OUOM',
 'EW_DIRECTION',
 'Y_OFFSET',
 'Y_OFFSET_OUOM',
 'NS_DIRECTION',
 'ROW_CHANGED_DATE',
 'GX_STATION_LATITUDE',
 'GX_STATION_LONGITUDE',
 'GX_CLOSURE']

We sort just to be sure, but probably don't need t0...

In [18]:
offsets = stations_data[['UWI', 'STATION_MD', 'Y_OFFSET', 'X_OFFSET', 'STATION_TVD']].sort_values(['UWI', 'STATION_MD'])

In [19]:
offsets.head(450)

UWI  STATION_MD  Y_OFFSET  X_OFFSET  STATION_TVD
0    300150011700        0.00      0.00      0.00         0.00
1    300150011700     1151.00      4.00     -0.38      1150.99
2    300150011700     1341.00      5.31     -0.59      1340.99
3    300150011700     1529.00      6.43     -0.79      1528.98
4    300150011700     1717.00      7.06     -0.94      1716.98
..            ...         ...       ...       ...          ...
445  429980051251    20499.96   6346.00   7965.24     10544.47
446  429980051251    20599.96   6410.91   8041.29     10545.97
447  429980051251    20699.96   6475.82   8117.35     10547.47
448  429980051251    20799.96   6540.74   8193.40     10548.97
449  429980051251    20899.96   6605.65   8269.46     10550.47

[450 rows x 5 columns]

In [20]:
tie_in_ids = offsets.groupby(['UWI'])['STATION_MD'].min()

In [21]:
tie_in_ids

UWI
300150011700    0.0
300150013401    0.0
429980051251    0.0
429991861452    0.0
Name: STATION_MD, dtype: float64

In [22]:
tie_ins = offsets.merge(tie_in_ids, on=['UWI', 'STATION_MD'])

In [23]:
tie_ins.head(450)

UWI  STATION_MD  Y_OFFSET  X_OFFSET  STATION_TVD
0  300150011700         0.0       0.0       0.0          0.0
1  300150013401         0.0       0.0       0.0          0.0
2  429980051251         0.0       0.0       0.0          0.0
3  429991861452         0.0       0.0       0.0          0.0

In [24]:
readings = stations_data[['UWI', 'STATION_MD', 'INCLINATION', 'AZIMUTH']]

In [25]:
readings[readings['UWI'] == '300150011700'].head(-5)

UWI  STATION_MD  INCLINATION  AZIMUTH
0   300150011700         0.0          0.0   359.93
1   300150011700      1151.0          0.4   354.63
2   300150011700      1341.0          0.4   346.83
3   300150011700      1529.0          0.3   353.73
4   300150011700      1717.0          0.1   324.13
..           ...         ...          ...      ...
86  300150011700      9206.0          3.7   257.33
87  300150011700      9238.0          3.8   259.53
88  300150011700      9300.0          4.4   259.73
89  300150011700      9360.0          4.1   261.53
90  300150011700      9423.0          4.0   263.33

[91 rows x 4 columns]

stats = [uwi, nlp_md]
    stats = np.append(stats, np.array(end_pts))
    stats = np.append(stats, best_fit_direction)
    stats = np.append(stats, [mean_inc, mean_az, mean_length])
    stats = np.append(stats, [lateral_length, tortuosity])
    stats = np.append(stats, [sinu, undu])

In [26]:
h_stats_names = h_stats_meta()['names']
empty_h_stats_rec = h_stats_meta()['empty_rec']

In [27]:
uwis = tie_ins[['UWI']]

In [28]:
#%%debug

stats = []
#pdb.set_trace()
for uwi in tie_ins['UWI']:
    survey = readings[readings['UWI'] == uwi][['STATION_MD', 'INCLINATION', 'AZIMUTH']].to_numpy()
    tie_in = tie_ins[tie_ins['UWI'] == uwi][['Y_OFFSET', 'X_OFFSET', 'STATION_TVD']].to_numpy()[0]
    try:
        print(uwi)
        #breakpoint()
        hs = h_stats(uwi, survey, tie_in, offset=None, nominal_step=5.0, min_inc=40.0, max_inc=87.0, critical_k=1.0/15000.0, discretize_step=1.0, mean_div=3)
        stats_dict = dict(zip(h_stats_names, hs))
        stats_dict['error'] = ''
        stats.append(stats_dict)
    except Exception as ex:
        stats_dict = dict(zip(h_stats_names, empty_h_stats_rec))
        stats_dict['uwi'] = uwi
        stats_dict['error'] = str(ex)
        stats.append(stats_dict)

300150011700
300150013401
429980051251
429991861452


In [29]:
stats

[{'uwi': '300150011700',
  'nlp_md': nan,
  'nlp_n': nan,
  'nlp_e': nan,
  'nlp_v': nan,
  'td_n': nan,
  'td_e': nan,
  'td_v': nan,
  'mean_bgn_n': nan,
  'mean_bgn_e': nan,
  'mean_bgn_v': nan,
  'mean_end_n': nan,
  'mean_end_e': nan,
  'mean_end_v': nan,
  'mean_dir_n': nan,
  'mean_dir_e': nan,
  'mean_dir_v': nan,
  'mean_inc': nan,
  'mean_azi': nan,
  'mean_len': nan,
  'lateral_len': nan,
  'tortuosity': nan,
  'sinuosity': nan,
  'undulation': nan,
  'error': 'No surveys have an inclination GT min: 40.0'},
 {'uwi': '300150013401',
  'nlp_md': nan,
  'nlp_n': nan,
  'nlp_e': nan,
  'nlp_v': nan,
  'td_n': nan,
  'td_e': nan,
  'td_v': nan,
  'mean_bgn_n': nan,
  'mean_bgn_e': nan,
  'mean_bgn_v': nan,
  'mean_end_n': nan,
  'mean_end_e': nan,
  'mean_end_v': nan,
  'mean_dir_n': nan,
  'mean_dir_e': nan,
  'mean_dir_v': nan,
  'mean_inc': nan,
  'mean_azi': nan,
  'mean_len': nan,
  'lateral_len': nan,
  'tortuosity': nan,
  'sinuosity': nan,
  'undulation': nan,
  'error': 

In [30]:
stats_pd = pd.DataFrame(stats)

In [31]:
stats_pd.head()

uwi              nlp_md               nlp_n              nlp_e  \
0  300150011700                 NaN                 NaN                NaN   
1  300150013401                 NaN                 NaN                NaN   
2  429980051251  10741.079999999998  -2.981817791085899  543.9752072196761   
3  429991861452  10743.149999999983  299.45059480724296   889.064476901142   

                nlp_v               td_n               td_e  \
0                 NaN                NaN                NaN   
1                 NaN                NaN                NaN   
2  10398.009672925853  7582.585306111008   9944.87159746175   
3  10399.964330733883  5471.869342999097  7313.147916906018   

                 td_v           mean_bgn_n         mean_bgn_e  ...  \
0                 NaN                  NaN                NaN  ...   
1                 NaN                  NaN                NaN  ...   
2   10579.33663923184  -2.9818177910841532  543.9752072196748  ...   
3  10399.964330733883   286.28781313504885  899.7067095512774  ...   

           mean_dir_e            mean_dir_v           mean_inc  \
0                 NaN                   NaN                NaN   
1                 NaN                   NaN                NaN   
2  0.7781554830098554  0.015009267966723534  89.14000000000001   
3  0.7776304995247264                   0.0               90.0   

             mean_azi            mean_len         lateral_len  \
0                 NaN                 NaN                 NaN   
1                 NaN                 NaN                 NaN   
2   51.10000000000001  12081.000000000018  12081.960000000003   
3  51.044135229021705   8247.574424279655   8250.650000000016   

           tortuosity           sinuosity undulation  \
0                 NaN                 NaN        NaN   
1                 NaN                 NaN        NaN   
2  1.0000794636205599                 1.0        1.0   
3  1.0003708487705956  1.0001707881916593        1.0   

                                         error  
0  No surveys have an inclination GT min: 40.0  
1  No surveys have an inclination GT min: 40.0  
2                                               
3                                               

[4 rows x 25 columns]

In [32]:
theta = np.radians(90)
c, s = np.cos(theta), np.sin(theta)
R = np.array(((c, -s), (s, c)))
print(R)

[[ 6.123234e-17 -1.000000e+00]
 [ 1.000000e+00  6.123234e-17]]


In [33]:
mutils.normalized([c, s])

array([6.123234e-17, 1.000000e+00])

In [34]:
np.linalg.norm([c, s])

1.0

In [35]:
R.shape

(2, 2)

In [36]:
x = np.array([[1,0,1],[1,0,1],[0,1,1],[0,1,1],[1,0,1],[1,0,1],[0,1,1],[0,1,1]])

In [37]:
x

array([[1, 0, 1],
       [1, 0, 1],
       [0, 1, 1],
       [0, 1, 1],
       [1, 0, 1],
       [1, 0, 1],
       [0, 1, 1],
       [0, 1, 1]])

In [38]:
x.shape

(8, 3)

In [39]:
R.dot(x[2,:2])

array([-1.000000e+00,  6.123234e-17])

In [40]:
xr = R.dot(x[:,:2].T).T

In [41]:
xr

array([[ 6.123234e-17,  1.000000e+00],
       [ 6.123234e-17,  1.000000e+00],
       [-1.000000e+00,  6.123234e-17],
       [-1.000000e+00,  6.123234e-17],
       [ 6.123234e-17,  1.000000e+00],
       [ 6.123234e-17,  1.000000e+00],
       [-1.000000e+00,  6.123234e-17],
       [-1.000000e+00,  6.123234e-17]])

In [42]:
y = np.array(x, copy=True)

In [43]:
y[:,:2] = xr

In [44]:
y

array([[ 0,  1,  1],
       [ 0,  1,  1],
       [-1,  0,  1],
       [-1,  0,  1],
       [ 0,  1,  1],
       [ 0,  1,  1],
       [-1,  0,  1],
       [-1,  0,  1]])

In [45]:
ggx_project_path = '\\GGX07-MID-PRD\Discovery-3bb3ef81-f9ed-4c1f-8e7b-5a2dab4ca8e2$\GGX007_DBBU'
oneuwi = '429991861452'
epgs_geo_27 = 4267
epsg_crs_27 = 32039
spcs_27 = 'SPCS27 - Texas Central'
epgs_geo_83 = 4269
epsg_crs_83 = 32139
spcs_27 = 'SPCS83 - Texas Central'
latlon = [31.08137, -103.46939]

In [46]:
# 31 43 21.85539(N) 103 49 14.83427(W)
#                     North         East     Units Scale Factor Converg.
# SPC TX C     - 3,233,113.731   369,538.808   MT  0.99996081   -1 47 46.5
lat_CC0554 = cvt.dms2decdeg(31, 43, 21.85539)
lon_CC0554 = cvt.dms2decdeg(-103, 49, 14.83427)
con_CC0554 = cvt.dms2decdeg(-1, 47, 46.5)

In [47]:
gc = GridConvergence(4269, 32139)

In [48]:
gc._prj._from_proj

Proj('+proj=longlat +datum=NAD83 +no_defs', preserve_units=True)

In [49]:
gc._prj._to_proj

Proj('+proj=lcc +lat_0=29.6666666666667 +lon_0=-100.333333333333 +lat_1=31.8833333333333 +lat_2=30.1166666666667 +x_0=700000 +y_0=3000000 +datum=NAD83 +units=m +no_defs', preserve_units=True)

In [50]:
gc.calc(np.array([[lat_CC0554, lon_CC0554]]))

array([-1.79624414])

In [51]:
con_CC0554

-1.7962500000000001

In [52]:
cvt.decdeg2dms(gc.calc(np.array([[lat_CC0554, lon_CC0554]])))

(array([-1.]), array([47.]), array([46.47890342]))

In [53]:
sql_stat_one = 'select * from WELL_DIR_SRVY_STATION WHERE UWI IN ({0})'.format("'429980051251'")
one_stat_dat = pandas.read_sql(sql_stat_one, cnxn)[['Y_OFFSET', 'X_OFFSET', 'STATION_TVD']].to_numpy()
one_stat_dat

array([[ 0.000000e+00,  0.000000e+00,  0.000000e+00],
       [ 0.000000e+00,  0.000000e+00,  1.000000e+02],
       [ 0.000000e+00,  0.000000e+00,  2.000000e+02],
       [ 0.000000e+00,  0.000000e+00,  3.000000e+02],
       [ 0.000000e+00,  0.000000e+00,  4.000000e+02],
       [ 0.000000e+00,  0.000000e+00,  5.000000e+02],
       [ 0.000000e+00,  0.000000e+00,  6.000000e+02],
       [ 0.000000e+00,  0.000000e+00,  7.000000e+02],
       [ 0.000000e+00,  0.000000e+00,  8.000000e+02],
       [ 0.000000e+00,  0.000000e+00,  9.000000e+02],
       [ 0.000000e+00,  0.000000e+00,  1.000000e+03],
       [ 0.000000e+00,  0.000000e+00,  1.100000e+03],
       [ 0.000000e+00,  0.000000e+00,  1.200000e+03],
       [ 0.000000e+00,  0.000000e+00,  1.300000e+03],
       [ 0.000000e+00,  0.000000e+00,  1.400000e+03],
       [ 0.000000e+00,  0.000000e+00,  1.500000e+03],
       [ 0.000000e+00,  0.000000e+00,  1.600000e+03],
       [ 0.000000e+00,  0.000000e+00,  1.700000e+03],
       [ 0.000000e+00,  0.00

In [54]:
one_stat_dat.shape

(236, 3)

In [55]:
sql_well_one = 'select * from WELL WHERE UWI IN ({0})'.format("'429980051251'")
one_well_dat = pandas.read_sql(sql_well_one, cnxn)[['SURFACE_LATITUDE', 'SURFACE_LONGITUDE']].to_numpy()
one_well_dat

array([[  31.00246, -103.42931]])

In [120]:
one_stat_nev = prj.toTrueNorthNEV(one_stat_dat, one_well_dat[0,0], one_well_dat[0,1], epgs_geo_27, epsg_crs_27, forward=False)

In [121]:
one_stat_nev

array([[ 0.00000000e+00, -0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -0.00000000e+00,  1.00000000e+02],
       [ 0.00000000e+00, -0.00000000e+00,  2.00000000e+02],
       [ 0.00000000e+00, -0.00000000e+00,  3.00000000e+02],
       [ 0.00000000e+00, -0.00000000e+00,  4.00000000e+02],
       [ 0.00000000e+00, -0.00000000e+00,  5.00000000e+02],
       [ 0.00000000e+00, -0.00000000e+00,  6.00000000e+02],
       [ 0.00000000e+00, -0.00000000e+00,  7.00000000e+02],
       [ 0.00000000e+00, -0.00000000e+00,  8.00000000e+02],
       [ 0.00000000e+00, -0.00000000e+00,  9.00000000e+02],
       [ 0.00000000e+00, -0.00000000e+00,  1.00000000e+03],
       [ 0.00000000e+00, -0.00000000e+00,  1.10000000e+03],
       [ 0.00000000e+00, -0.00000000e+00,  1.20000000e+03],
       [ 0.00000000e+00, -0.00000000e+00,  1.30000000e+03],
       [ 0.00000000e+00, -0.00000000e+00,  1.40000000e+03],
       [ 0.00000000e+00, -0.00000000e+00,  1.50000000e+03],
       [ 0.00000000e+00, -0.00000000e+00

In [122]:
cvt.usft2meters(one_stat_nev)

array([[ 0.00000000e+00, -0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -0.00000000e+00,  3.04800610e+01],
       [ 0.00000000e+00, -0.00000000e+00,  6.09601219e+01],
       [ 0.00000000e+00, -0.00000000e+00,  9.14401829e+01],
       [ 0.00000000e+00, -0.00000000e+00,  1.21920244e+02],
       [ 0.00000000e+00, -0.00000000e+00,  1.52400305e+02],
       [ 0.00000000e+00, -0.00000000e+00,  1.82880366e+02],
       [ 0.00000000e+00, -0.00000000e+00,  2.13360427e+02],
       [ 0.00000000e+00, -0.00000000e+00,  2.43840488e+02],
       [ 0.00000000e+00, -0.00000000e+00,  2.74320549e+02],
       [ 0.00000000e+00, -0.00000000e+00,  3.04800610e+02],
       [ 0.00000000e+00, -0.00000000e+00,  3.35280671e+02],
       [ 0.00000000e+00, -0.00000000e+00,  3.65760732e+02],
       [ 0.00000000e+00, -0.00000000e+00,  3.96240792e+02],
       [ 0.00000000e+00, -0.00000000e+00,  4.26720853e+02],
       [ 0.00000000e+00, -0.00000000e+00,  4.57200914e+02],
       [ 0.00000000e+00, -0.00000000e+00

In [123]:
one_stat_ll = cvt.lmp(cvt.usft2meters(one_stat_nev), one_well_dat[0,0], one_well_dat[0,1], datum='nad27')
one_stat_ll

array([[  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -103.42931   ],
       [  31.00246   , -1

In [124]:
sql_ll_one = 'select * from WELL_DIR_SRVY_STATION WHERE UWI IN ({0})'.format("'429980051251'")
one_ll_ggx = pandas.read_sql(sql_ll_one, cnxn)[['GX_STATION_LATITUDE', 'GX_STATION_LONGITUDE']].to_numpy()
one_ll_ggx

array([[  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  31.00246, -103.42931],
       [  

In [125]:
one_stat_ll - one_ll_ggx

array([[ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.

In [126]:
(one_stat_ll - one_ll_ggx).max()

0.0007502965498424885

In [127]:
0.0007502965498424885

0.0007502965498424885

In [128]:
cvt.decdeg2dms((one_stat_ll - one_ll_ggx).max())

(0.0, 0.0, 2.7010675794329586)

In [129]:
(0.0, 0.0, 2.7010675794329586)

(0.0, 0.0, 2.7010675794329586)

In [130]:
tvds_geo = np.vstack([one_stat_ll[-1,:], one_ll_ggx[-1:]])
tvds_geo

array([[  31.0248003 , -103.39894127],
       [  31.02405   , -103.39824   ]])

In [131]:
tvds_geo.shape

(2, 2)

In [132]:
tvds_crt = prj.Project(epgs_geo_27, epsg_crs_27).forward(tvds_geo)

In [133]:
tvds_crt

array([[ 507161.30101425, 1039905.88380368],
       [ 506882.47729755, 1040117.93675483]])

In [134]:
tvds_crt[0]

array([ 507161.30101425, 1039905.88380368])

In [135]:
np.linalg.norm(tvds_crt[0] - tvds_crt[1])

350.2986141670411

In [136]:
350.2986141670411

350.2986141670411

In [140]:
ggx_lat_crs = prj.Project(epgs_geo_27, epsg_crs_27).forward(np.array([[one_well_dat[0,0], one_well_dat[0,1]]]))

In [141]:
ggx_lat_crs

array([[ 499302.91791636, 1030171.28216258]])

In [146]:
one_stat_dat[-1,:1]

array([7853.94])

In [149]:
ggx_lat_crs[0] + one_stat_dat[-1,:1]

array([ 507156.85791636, 1038025.22216258])

In [151]:
prj.Project(epgs_geo_27, epsg_crs_27).inverse([ggx_lat_crs[0] + one_stat_dat[-1,:1]])

array([[  31.02464546, -103.40494284]])

In [152]:
one_ll_ggx[-1]

array([  31.02405, -103.39824])